In [16]:
import pandas as pd
import numpy as np
import scipy.io as spio
import plotly.express as px
from functools import reduce
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Casos cofirmados por INS 
## A la fecha 26/07/2020  - 2 pm

In [17]:
#Base de datos
df_covid = pd.read_csv('Datos/Casos_positivos_de_COVID-19_en_Colombia.csv', delimiter=",", decimal=".") #Datos de COVID por INS

#Renombar las columnas y manejo ciertas varibales
df_covid.rename(columns={"Fecha de muerte":'dateM'}, inplace=True)
df_covid[["dateMuerte","timeEl"]]=df_covid.dateM.str.split("T",expand=True) #INS coloca la fecha y la hora, se separa la fecha

df_covid.rename(columns={"fecha reporte web":'date0'}, inplace=True)
df_covid[["date","timeE"]]=df_covid.date0.str.split("T",expand=True) #INS coloca la fecha y la hora, se separa la fecha
df_covid["casos"] = 1

C:\Users\Pc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning:

Columns (10,19,20) have mixed types.Specify dtype option on import or set low_memory=False.



Casos confirmados a nivel departemental y Nacional

In [18]:
#Casos por departamentos a la fecha
covid_dep= df_covid.groupby(['Codigo departamento',])['casos'].sum() 
df_covid_depf=covid_dep.reset_index()
df_covid_depf.to_excel("casospordepartamentoa0708.xlsx")

#Casos por departamentos por fecha de reporte
covid_dep0= df_covid.groupby([ "date",'Codigo departamento',])['casos'].sum() 
df_covid_dep=covid_dep0.reset_index()
df_covid_dep.to_excel("casospordepartamento.xlsx")

#Casos a nivel nacional por fecha de reporte
covid_nac= df_covid.groupby([ "date"])['casos'].sum() 
df_covid_nac=covid_nac.reset_index()




Fallecidos a a nivel departemental y Nacional

In [19]:
df_covid = df_covid[df_covid.dateMuerte.notnull()]

#Fallecidos por departamento y fecha
covid_depM= df_covid.groupby(['Codigo departamento',"dateMuerte"])['casos'].sum() 
df_covid_depM=covid_depM.reset_index()

#Total de fallecido por departamento a la fecha
covid_depMf= df_covid.groupby(['Codigo departamento'])['casos'].sum() 
df_covid_depMf=covid_depMf.reset_index()

#Fallecidos Nacional y fecha
covid_NacM= df_covid.groupby(["dateMuerte"])['casos'].sum() 
df_covid_NacM=covid_NacM.reset_index()

## Casos a nivel nacional 

In [20]:

fig = make_subplots(specs=[[{"secondary_y": True}]],)

fig.add_trace(go.Scatter(
                x=df_covid_nac.date,
                y=df_covid_nac['casos'],
                name="Casos COVID-19 confirmados",
                line_color='Blue',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_covid_NacM.dateMuerte,
                y=df_covid_NacM['casos'],
                name="Fallecidos",
                line_color='Orange',
                opacity=1), secondary_y=True,)
# Titulo
fig.update_layout(
    title_text="Casos diarios COVID-19 confirmados", 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
   
)
# Set x-axis title

#title_text="<b>Periodo mensual ( Enero 2014 - Enero 2020)</b>"

fig.update_xaxes(gridcolor='#bdbdbd')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Casos Confirmados</b>", secondary_y=False,gridcolor='#bdbdbd',zerolinecolor='#bdbdbd')
fig.update_yaxes(title_text="<b>Fallecidos</b>", secondary_y=True)


fig.show()


## Casos Departamentos

In [6]:
df_dane = pd.read_csv('Datos/coddane.csv', delimiter=";", decimal=",") #Nombres de los departamentos
data_frames= [df_covid_depf,df_dane,df_covid_depMf]
df_covid_depf2 = reduce(lambda  left,right: pd.merge(left,right,on=["Codigo departamento"],
                                            how='outer'), data_frames)

In [21]:
fig = go.Figure([go.Bar(x=df_covid_depf2['Departamentos'], y=df_covid_depf2['casos_x'])])

fig.update_layout(
    title_text="Casos acumulados por COVID-19 confirmados", 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
   
)

fig.update_yaxes(title_text="<b>Casos Confirmados</b>",gridcolor='#bdbdbd',zerolinecolor='#bdbdbd')
fig.show()

In [22]:
fig = go.Figure([go.Bar(x=df_covid_depf2['Departamentos'], y=df_covid_depf2['casos_y'])])

fig.update_layout(
    title_text="Fallecidos acumulados por COVID-19- Departamentos", 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
   
)

fig.update_yaxes(title_text="<b>Casos Confirmados</b>",gridcolor='#bdbdbd',zerolinecolor='#bdbdbd')
fig.show()

## Antioquia

In [9]:
df_covid_Antioquia = df_covid_dep[ df_covid_dep["Codigo departamento"] == 5]
df_covid_Antioquiafallecidos = df_covid_depM[ df_covid_depM["Codigo departamento"] == 5]


fig = make_subplots(specs=[[{"secondary_y": True}]],)

fig.add_trace(go.Scatter(
                x=df_covid_Antioquia.date,
                y=df_covid_Antioquia['casos'],
                name="Casos COVID-19 confirmados",
                line_color='Blue',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_covid_Antioquiafallecidos.dateMuerte,
                y=df_covid_Antioquiafallecidos['casos'],
                name="Fallecidos",
                line_color='Orange',
                opacity=1), secondary_y=True,)
# Titulo
fig.update_layout(
    title_text="Casos diarios COVID-19 confirmados en Antioquia", 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
   
)
# Set x-axis title

#title_text="<b>Periodo mensual ( Enero 2014 - Enero 2020)</b>"

fig.update_xaxes(gridcolor='#bdbdbd')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Casos Confirmados</b>", secondary_y=False,gridcolor='#bdbdbd',zerolinecolor='#bdbdbd')
fig.update_yaxes(title_text="<b>Fallecidos</b>", secondary_y=True)


fig.show()

# Datos de Movilidad de Google

In [10]:
#Base de datos
df_google = pd.read_csv('Datos/Global_Mobility_Report.csv', delimiter=",", decimal=".") #Datos de Google

C:\Users\Pc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning:

Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.



In [11]:
#Filtrar datos de Colombia
df_g_col = df_google[ df_google["country_region"] == "Colombia" ]
#Llenar los espacios vacios de sub_region_1 ( los espacios vacios son los datos a nivel nacional)
df_g_col["sub_region_1"].fillna("Nacional", inplace = True) 

C:\Users\Pc\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
df_g_colN=df_g_col[ df_g_col["sub_region_1"] == "Nacional" ]

fig = make_subplots(specs=[[{"secondary_y": True}]],)

fig.add_trace(go.Scatter(
                x=df_g_colN.date,
                y=df_g_colN['retail_and_recreation_percent_change_from_baseline'],
                name="Tiendas y ocio",
                line_color='Blue',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_g_colN.date,
                y=df_g_colN['grocery_and_pharmacy_percent_change_from_baseline'],
                name="Supermercados y farmacias",
                line_color='black',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_g_colN.date,
                y=df_g_colN['workplaces_percent_change_from_baseline'],
                name="Lugares de trabajo",
                line_color='green',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_g_colN.date,
                y=df_g_colN['residential_percent_change_from_baseline'],
                name="Zonas residenciales",
                line_color='orange',
                opacity=1), secondary_y=False,)


# Titulo
fig.update_layout(
    title_text="Cambios en la movilidad - Nivel Nacional", 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
   
)
# Set x-axis title

#title_text="<b>Periodo mensual ( Enero 2014 - Enero 2020)</b>"

fig.update_xaxes(gridcolor='#bdbdbd')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Cambio porcentual versus el valor de referencia</b>", secondary_y=False,gridcolor='#bdbdbd',zerolinecolor='#bdbdbd')



fig.show()


## Casos confirmados y Movilidad de Google

Nivel Nacional

In [13]:
fig = make_subplots(specs=[[{"secondary_y": True}]],)

fig.add_trace(go.Scatter(
                x=df_g_colN.date,
                y=df_g_colN['retail_and_recreation_percent_change_from_baseline'],
                name="Tiendas y ocio",
                line_color='Blue',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_g_colN.date,
                y=df_g_colN['grocery_and_pharmacy_percent_change_from_baseline'],
                name="Supermercados y farmacias",
                line_color='black',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_g_colN.date,
                y=df_g_colN['workplaces_percent_change_from_baseline'],
                name="Lugares de trabajo",
                line_color='green',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_covid_nac.date,
                y=df_covid_nac['casos'],
                name="Casos COVID-19 confirmados",
                line_color='red',
                opacity=1), secondary_y=True,)

# Titulo
fig.update_layout(
    title_text="Cambios en la movilidad - Nivel Nacional", 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
   
)
# Set x-axis title

#title_text="<b>Periodo mensual ( Enero 2014 - Enero 2020)</b>"

fig.update_xaxes(gridcolor='#bdbdbd')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Cambio porcentual versus el valor de referencia</b>", secondary_y=False,gridcolor='#bdbdbd',zerolinecolor='#bdbdbd')

fig.update_yaxes(title_text="<b>Casos Diarios</b>", secondary_y=True)

fig.show()

## Casos confirmados y Movilidad de Google

Antioquia

In [14]:
df_g_colA=df_g_col[ df_g_col["sub_region_1"] == "Antioquia" ]

fig = make_subplots(specs=[[{"secondary_y": True}]],)

fig.add_trace(go.Scatter(
                x=df_g_colA.date,
                y=df_g_colA['retail_and_recreation_percent_change_from_baseline'],
                name="Tiendas y ocio",
                line_color='Blue',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_g_colA.date,
                y=df_g_colA['grocery_and_pharmacy_percent_change_from_baseline'],
                name="Supermercados y farmacias",
                line_color='Orange',
                opacity=1), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_g_colA.date,
                y=df_g_colA['workplaces_percent_change_from_baseline'],
                name="Lugares de trabajo",
                line_color='green',
                opacity=0.6), secondary_y=False,)

fig.add_trace(go.Scatter(
                x=df_covid_Antioquia.date,
                y=df_covid_Antioquia['casos'],
                name="Casos COVID-19 confirmados",
                line_color='black',
                opacity=1), secondary_y=True,)

# Titulo
fig.update_layout(
    title_text="Cambios en la movilidad - Antioquia", 
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
   
)
# Set x-axis title

#title_text="<b>Periodo mensual ( Enero 2014 - Enero 2020)</b>"

fig.update_xaxes(gridcolor='#bdbdbd')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Cambio porcentual versus el valor de referencia</b>", secondary_y=False,gridcolor='#bdbdbd',zerolinecolor='#bdbdbd')

fig.update_yaxes(title_text="<b>Casos Diarios</b>", secondary_y=True)

fig.show()

# Exportar datos a nivel nacional a Excel 

In [15]:
df_covid_NacM2 = df_covid_NacM.rename(columns = {"dateMuerte": "date",
                                                "casos":"fallecidos"}) 
df_covid_nac2 = df_covid_nac.rename(columns = {"casos": "confirmados"}) 


data_frames= [df_covid_nac2,df_covid_NacM2]
df_covid_NacTotal = reduce(lambda  left,right: pd.merge(left,right,on=["date"],
                                            how='outer'), data_frames)
df_covid_NacTotal.to_excel("CasosNacionalporfecha.xlsx")
